In [1]:
import json
from glob import glob
from multiprocessing import Pool

from tqdm import tqdm

In [2]:
train_annos = '/data/DeepFashion2/train/annos/*.json'
vlidation_annos = '/data/DeepFashion2/validation/annos/*.json'
# test_annos = '/data/DeepFasion2/test/annos/*.json'

In [3]:
def count_items(json_filepath: str) -> int:
    with open(json_filepath) as f:
        data = json.load(f)
    num_items = len([item for item in data if item.startswith('item')])
    return json_filepath, num_items


def find_max_items(json_path):
    json_files = glob(json_path)
    max_items = 0
    max_item_filepath = ''
    with Pool(10) as p:
        for filepath, num_items in tqdm(p.imap_unordered(count_items, json_files), total=len(json_files)):
            if num_items > max_items:
                max_items = num_items
                max_item_filepath = filepath
    return max_items, max_item_filepath

In [4]:
find_max_items(vlidation_annos)

100%|██████████| 32153/32153 [00:00<00:00, 46457.64it/s]


(5, '/data/DeepFashion2/validation/annos/005592.json')

In [5]:
find_max_items(train_annos)

100%|██████████| 191961/191961 [00:02<00:00, 66378.47it/s]


(8, '/data/DeepFashion2/train/annos/095484.json')